In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from IPython.display import display

from carbon_bombs.conf import REPO_PATH
from carbon_bombs.conf import FPATH_OUT_BANK
from carbon_bombs.conf import FPATH_OUT_CB
from carbon_bombs.conf import FPATH_OUT_COMP
from carbon_bombs.conf import FPATH_OUT_CONX_BANK_COMP
from carbon_bombs.conf import FPATH_OUT_CONX_CB_COMP
from carbon_bombs.conf import FPATH_OUT_COUNTRY


def compare_dataframes(df1, df2, key_col):
    
    if not isinstance(key_col, list):
        df1 = df1.loc[(~df1[key_col].isna()) & (df1[key_col] != "None")]
        df2 = df2.loc[~df2[key_col].isna() & (df2[key_col] != "None")]
        key_col = [key_col]
        
        
    df1["key_col"] = df1.apply(lambda x: (" - ".join([str(x[k]) for k in key_col])), axis=1)
    df2["key_col"] = df2.apply(lambda x: (" - ".join([str(x[k]) for k in key_col])), axis=1)

    df1 = df1.sort_values(by="key_col").reset_index(drop=True).fillna("None")
    df2 = df2.sort_values(by="key_col").reset_index(drop=True).fillna("None")
    
    diff = set(df1.columns) - set(df2.columns)
    if diff:
        print(f"New df has new columns:", diff)

    diff = set(df2.columns) - set(df1.columns)
    if diff:
        print(f"New df misses columns:", diff)
        
    uniq_keys1 = set(df1["key_col"])
    uniq_keys2 = set(df2["key_col"])
        
    if uniq_keys1.symmetric_difference(uniq_keys2):
        if uniq_keys1 - uniq_keys2:
            print("Found new keys:", uniq_keys1 - uniq_keys2)
        if uniq_keys2 - uniq_keys1:
            print("Missing keys:", uniq_keys2 - uniq_keys1)
        
        common_keys = uniq_keys1.intersection(uniq_keys2)
        df1 = df1.loc[df1["key_col"].isin(common_keys)].drop_duplicates().reset_index(drop=True)
        df2 = df2.loc[df2["key_col"].isin(common_keys)].drop_duplicates().reset_index(drop=True)

    full_comp = []
        
    for col in df2.columns:
        if col == "key_col":
            continue
        if col not in df1:
            print(f"NF // {col}  // not in new_df")
            continue
    
        if col not in key_col:
            df1_ = df1.set_index(key_col).copy()
            df2_ = df2.set_index(key_col).copy()
        else:
            df1_ = df1
            df2_ = df2
            
        comp = df1_[col].fillna("None").replace("", "None").compare(
            df2_[col].fillna("None").replace("", "None"),
            result_names=("new", "old")
        ).reset_index()
        comp["column"] = col
        
        comp["key_col"] = ""

        if "index" not in comp:
            if len(comp):
                comp["key_col"] = comp.apply(lambda x: (" - ".join([str(x[k]) for k in key_col])), axis=1)
        
        
        if len(comp) == 0:
            print(f"OK -- {col} -- ALL OK")
        else:
            print(f"KO -- {col} -- NOT OK (n={len(comp)})")
            # display(comp)
            
        full_comp += [comp]
        
    full_comp = pd.concat(full_comp).reset_index(drop=True)
    
    return full_comp

def compare_cleaned_datasets(old_data_path="data_cleaned_SAVE"):
    
    cleaned_datasets_fpaths = [
        FPATH_OUT_CB,
        FPATH_OUT_COMP,
        FPATH_OUT_BANK,
        FPATH_OUT_CONX_BANK_COMP,
        FPATH_OUT_CONX_CB_COMP,
        FPATH_OUT_COUNTRY,
    ]
    
    key_cols_map = {
        "carbon_bombs_data.csv": ["Carbon_bomb_name_source_CB", "Country_source_CB"],
        "company_data.csv": "Company_name",
        "bank_data.csv": "Bank Name",
        "connection_bank_company.csv": ["Bank", "Company"],
        "connection_carbonbombs_company.csv": ["Carbon_bomb_name", "Company", "Country"],
        "country_data.csv": "Country_source_CB",
    }
    
    full_comp = []
    
    for fpath in cleaned_datasets_fpaths:
        fname = fpath.split("/")[-1]
        save_path = f"{REPO_PATH}/{old_data_path}/{fname}"
        
        new_df = pd.read_csv(fpath)
        old_df = pd.read_csv(save_path)
        
        print(fname)
        comp = compare_dataframes(new_df, old_df, key_cols_map[fname])
        comp["file"] = fname
        comp = comp[["file", "column", "key_col", "new", "old"]]
        full_comp += [comp]
        
    full_comp = pd.concat(full_comp).reset_index(drop=True)
    
    return full_comp

In [3]:
full_comp = compare_cleaned_datasets("data_cleaned copy 2//")

carbon_bombs_data.csv
OK -- Carbon_bomb_name_source_CB -- ALL OK
OK -- Country_source_CB -- ALL OK
OK -- World_region -- ALL OK
OK -- Potential_GtCO2_source_CB -- ALL OK
OK -- Fuel_type_source_CB -- ALL OK
KO -- GEM_id_source_GEM -- NOT OK (n=2)
KO -- GEM_url_source_GEM -- NOT OK (n=2)
KO -- Latitude -- NOT OK (n=115)
KO -- Longitude -- NOT OK (n=121)
KO -- Latitude_longitude_source -- NOT OK (n=83)
KO -- Operators_source_GEM -- NOT OK (n=2)
KO -- Parent_company_source_GEM -- NOT OK (n=2)
KO -- Companies_involved_source_GEM -- NOT OK (n=2)
KO -- GEM_project_name_source_GEM -- NOT OK (n=2)
KO -- Carbon_bomb_description -- NOT OK (n=2)
OK -- Carbon_bomb_start_year -- ALL OK
OK -- Status_source_CB -- ALL OK
KO -- Status_source_GEM -- NOT OK (n=2)
OK -- Status_lvl_1 -- ALL OK
OK -- Status_lvl_2 -- ALL OK
company_data.csv
Found new keys: {'Texas American Resources Company', 'Auterra Operating', 'Recoil Resources Operating', 'Ageron Energy', 'SilverBow Resources Inc', 'Virtex Operating Compa

In [4]:
full_comp.shape

(352, 5)

In [5]:
full_comp.head(50)

,file,column,key_col,new,old
0,carbon_bombs_data.csv,GEM_id_source_GEM,Anadarko Shelf_Oklahoma - United States,No informations available on GEM,OG0014204|OG0014195|OG0014202|OG0014200|OG0014...
1,carbon_bombs_data.csv,GEM_id_source_GEM,Eagle Ford Shale - United States,OG0014242|OG0014613|OG0014174|OG0014360|OG0014...,No informations available on GEM
2,carbon_bombs_data.csv,GEM_url_source_GEM,Anadarko Shelf_Oklahoma - United States,No informations available on GEM,https://www.gem.wiki/Friesian_(GC599)_-_Anadar...
3,carbon_bombs_data.csv,GEM_url_source_GEM,Eagle Ford Shale - United States,https://www.gem.wiki/Aguila_Vado_(Eagleford)_-...,No informations available on GEM
4,carbon_bombs_data.csv,Latitude,Ahwaz Asmari - Iran,31.307133,31.320649
5,carbon_bombs_data.csv,Latitude,Ahwaz Bangestan - Iran,31.275164,31.279267
6,carbon_bombs_data.csv,Latitude,Anadarko Shelf_Oklahoma - United States,31.7908,27.327
7,carbon_bombs_data.csv,Latitude,Arctic LNG 2 T1-3 - Russia,69.7,64.762392
8,carbon_bombs_data.csv,Latitude,Asab - United Arab Emirates,23.613991,23.498818
9,carbon_bombs_data.csv,Latitude,Bab (Gasco) - United Arab Emirates,24.000249,23.906394


In [6]:
full_comp.to_csv("comparison.csv", index=False)